# Initalize the client

In [20]:
from dotenv import load_dotenv

load_dotenv()

True

In [22]:
import os
from astrapy.db import AstraDB

# Initialize the client. The namespace parameter is optional if you use
# "default_keyspace".
db = AstraDB(
    token=os.environ['ASTRA_DB_APPLICATION_TOKEN'],
    api_endpoint=os.environ['ASTRA_DB_API_ENDPOINT'],
    namespace="default_keyspace",
)
print(db)

Astra DB[endpoint="https://0333559e-a6bc-49f8-a072-9b15069bb6eb-us-east1.apps.astra.datastax.com"]


# Create a collection

In [28]:
# Create a collection
collection = db.collection('collection_name=midas_db')
print(collection)

Astra DB Collection[name="collection_name=midas_db", endpoint="https://0333559e-a6bc-49f8-a072-9b15069bb6eb-us-east1.apps.astra.datastax.com"]


Here, we're setting up the OpenAI API key and initializing a `SimpleNodeParser`. This parser processes our list of `Document` objects into 'nodes', which are the basic units that `llama_index` uses for indexing and querying. The first node is displayed below.

In [19]:
import os

os.environ['OPENAI_API_KEY'] = '<your OpenAI API key>'  # platform.openai.com

In [20]:
from llama_index.node_parser import SimpleNodeParser

parser = SimpleNodeParser()

nodes = parser.get_nodes_from_documents(docs)
nodes[0]

Node(text='Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.', doc_id='8968aa3c-abbe-4fe0-ad91-be0298830542', embedding=None, doc_hash='ba8eb7d843763d4f7e7f71f0be64c70c4597fd712bfcce77007fc4c821b8d1b6', extra_info={'title': 'University_of_Notre_Dame'}, node_info={'start': 0, 'end': 695}, relationships={<DocumentRelationship.SOURCE: '1'>: '5733be284776

In [21]:
len(nodes)

18892

### Indexing in Pinecone

Pinecone is a managed vector database service designed for machine learning applications. We're using it in this context to store and retrieve embeddings generated by our language model, enabling efficient and scalable semantic similarity-based search.

We initialize Pinecone with the relevant API key and environment that we [get for **free** in the console](https://app.pinecone.io/), then create a new index. The index has a dimension of 1536 and uses cosine similarity, which is the recommended metric for comparing vectors produced by the `text-embedding-ada-002` model we'll be using.

In [ ]:
from pinecone import Pinecone
from pinecone import ServerlessSpec


# find API key in console at app.pinecone.io
os.environ['PINECONE_API_KEY'] = '<your API key here>'

# initialize connection to pinecone
pc = Pinecone(api_key=os.getenv('PINECONE_API_KEY'))


# create the index if it does not exist already
index_name = 'llama-index-intro'
existing_indexes = [i.get('name') for i in pc.list_indexes()]

if index_name not in existing_indexes:
    pc.create_index(
        name=index_name,
        dimension=1536,
        metric='cosine',
        spec=ServerlessSpec(cloud='aws', region='us-west-2')
    )

# connect to the index
pinecone_index = pc.Index(index_name)

Here, we're initializing a `PineconeVectorStore` with our previously created Pinecone index. This object will serve as the storage and retrieval interface for our document embeddings in Pinecone's vector database.

In [23]:
from llama_index.vector_stores import PineconeVectorStore

# we can select a namespace (acts as a partition in an index)
namespace = '' # default namespace

vector_store = PineconeVectorStore(pinecone_index=pinecone_index)

Next we initialize the `GPTVectorStoreIndex` with our list of `Document` objects, using the `PineconeVectorStore` as storage and `OpenAIEmbedding` model for embeddings.

`StorageContext` is used to configure the storage setup, and `ServiceContext` sets up the embedding model. The `GPTVectorStoreIndex` handles the indexing and querying process, making use of the provided storage and service contexts.

In [24]:
from llama_index import GPTVectorStoreIndex, StorageContext, ServiceContext
from llama_index.embeddings.openai import OpenAIEmbedding

# setup our storage (vector db)
storage_context = StorageContext.from_defaults(
    vector_store=vector_store
)
# setup the index/query process, ie the embedding model (and completion if used)
embed_model = OpenAIEmbedding(model='text-embedding-ada-002', embed_batch_size=100)
service_context = ServiceContext.from_defaults(embed_model=embed_model)

index = GPTVectorStoreIndex.from_documents(
    docs, storage_context=storage_context,
    service_context=service_context
)

Finally we can build a query engine from the `index` we build and use this engine to perform a query.

In [25]:
query_engine = index.as_query_engine()
res = query_engine.query("in what year was the college of engineering established at the University of Notre Dame?")
print(res)


The College of Engineering was established in 1920.


That's our quick intro to using Llama-index and Pinecone! Once we're done testing the system we should delete the Pinecone index to save resources:

In [26]:
pc.delete_index(index_name)

---